In [ ]:
from web3 import Web3
import json
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# -----------------------------
# CONFIGURATION
# -----------------------------
RPC_URL = os.getenv("GNOSIS_RPC")
CONTRACT_ADDRESS = "0xC05e7412439bD7e91730a6880E18d5D5873F632C"

CONTRACT_ABI = json.loads(
    """
[
    {
        "inputs": [],
        "name": "numUndeliveredRequests",
        "outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}],
        "stateMutability": "view",
        "type": "function"
    },
    {
        "inputs": [
            {"internalType": "uint256", "name": "size", "type": "uint256"},
            {"internalType": "uint256", "name": "offset", "type": "uint256"}
        ],
        "name": "getUndeliveredRequestIds",
        "outputs": [
            {"internalType": "bytes32[]", "name": "", "type": "bytes32[]"}
        ],
        "stateMutability": "view",
        "type": "function"
    }
]
"""
)

w3 = Web3(Web3.HTTPProvider(RPC_URL))
if not w3.is_connected():
    raise Exception("Could not connect to RPC")

contract = w3.eth.contract(address=CONTRACT_ADDRESS, abi=CONTRACT_ABI)


def get_all_undelivered_requests(batch_size=100):
    total = contract.functions.numUndeliveredRequests().call()
    all_ids = []
    offset = 0

    while offset < total:
        size = min(batch_size, total - offset)  # don't exceed total
        ids = contract.functions.getUndeliveredRequestIds(size, offset).call()
        all_ids.extend(ids)
        offset += len(ids)

    return all_ids


if __name__ == "__main__":
    all_ids = get_all_undelivered_requests(batch_size=100)
    print(f"✅ Total undelivered request IDs: {len(all_ids)}")

In [ ]:
[i.hex() for i in all_ids]